In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import numpy as np
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import os
import keras_tuner as kt
import cv2
import pandas as pd

In [3]:
# with zipfile.ZipFile('/content/drive/MyDrive/robot model.zip', 'r') as zip_ref:
#   zip_ref.extractall()

# !unzip '/content/drive/MyDrive/robot model.zip' -d "/content/drive/My Drive/robotdata/"

# Importing

In [4]:
image_folder_path = "/content/drive/My Drive/robotdata/all/"
image_folder_path2 = "/content/drive/My Drive/robotdata/all2"
# Get a list of all the images in the folder
image_files = os.listdir(image_folder_path)
image_files2 = os.listdir(image_folder_path2)

input = pd.DataFrame(image_files)

cart = []
for x in range(2500):
  arr = []
  for i in range(len(image_files[x].split('-'))):
        if (image_files[x].split('-')[i]) != '':
            arr.append(image_files[x].split('-')[i])
  cart.append(np.array([float(arr[7]), float(arr[8]), float(arr[9]), float(arr[10]), float(arr[11]), float(arr[12]), float(arr[13].split(".jpg")[0]) ]))
output = pd.DataFrame(cart)

In [5]:
output.head()

,0,1,2,3,4,5,6
0,477.25,12.90,622.31,0.614,0.371,0.440,0.540
1,481.13,12.88,621.63,0.617,0.365,0.435,0.544
2,484.97,12.85,620.80,0.621,0.360,0.430,0.548
3,496.12,12.77,617.93,0.630,0.344,0.416,0.559
4,488.74,12.82,619.93,0.624,0.355,0.425,0.552


In [6]:
df = [input,output]
df = pd.concat(df)
df = pd.concat([input, output], axis=1, join='inner')
df.columns = ['X', 'Y1', "Y2", 'Y3', "Y4", 'Y5', "Y6", 'Y7']

In [7]:
df.head()

,X,Y1,Y2,Y3,Y4,Y5,Y6,Y7
0,-2.0-23.0--21.0--8.01--21.99-90.0-imgcap17864-...,477.25,12.90,622.31,0.614,0.371,0.440,0.540
1,-2.0-24.0--22.01--8.01--23.0-90.0-imgcap17865-...,481.13,12.88,621.63,0.617,0.365,0.435,0.544
2,-2.0-25.0--23.0--8.01--24.0-90.0-imgcap17866-4...,484.97,12.85,620.80,0.621,0.360,0.430,0.548
3,-2.0-28.0--26.0--8.01--26.99-90.0-imgcap17869-...,496.12,12.77,617.93,0.630,0.344,0.416,0.559
4,-2.0-26.0--24.0--8.01--25.0-90.0-imgcap17867-4...,488.74,12.82,619.93,0.624,0.355,0.425,0.552


In [8]:
images = []
count = 1
for x in df['X']:
  if count > 2500:
    break
  else:
    img = np.resize(np.asarray(Image.open(image_folder_path+x), dtype='float32'), (128,128,3)) / 255
    images.append(img)
    count+=1


In [9]:
y = np.array([df['Y1'], df["Y2"], df['Y3'], df["Y4"], df['Y5'], df["Y6"], df['Y7']]).T
X_train, X_test, y_train, y_test = train_test_split(images, y, random_state=0, train_size = 0.8)

# Defining the model

In [63]:
def model_builder(hp):
    act = hp.Choice("activation7", ["relu", "tanh", "sigmoid"])
    model = tf.keras.models.Sequential()

    hp_units_conv1 = hp.Int('convunits1', min_value=32, max_value=128, step=16)
    model.add(tf.keras.layers.Conv2D(hp_units_conv1, (3, 3), activation=act, input_shape=(128,128,3)))
    model.add(tf.keras.layers.MaxPooling2D(2,2))

    hp_units_conv2 = hp.Int('convunits2', min_value=16, max_value=128, step=16)
    model.add(tf.keras.layers.Conv2D(hp_units_conv2, (3, 3), activation=act))
    model.add(tf.keras.layers.MaxPooling2D(2,2))

    hp_units_conv3 = hp.Int('convunits3', min_value=16, max_value=256, step=16)
    model.add(tf.keras.layers.Conv2D(hp_units_conv3, (3, 3), activation=act))
    model.add(tf.keras.layers.MaxPooling2D(2,2))
    model.add(tf.keras.layers.Flatten())

    # hp_units_conv4 = hp.Int('convunits4', min_value=16, max_value=128, step=16)
    # model.add(tf.keras.layers.Conv2D(hp_units_conv4, (3, 3), activation=act))
    # model.add(tf.keras.layers.MaxPooling2D(2,2))

    # hp_units_conv5 = hp.Int('convunits5', min_value=16, max_value=128, step=16)
    # model.add(tf.keras.layers.Conv2D(hp_units_conv5, (3, 3), activation=act))
    # model.add(tf.keras.layers.MaxPooling2D(2,2))

    # hp_units_conv3 = hp.Int('convunits6', min_value=16, max_value=256, step=16)
    # model.add(tf.keras.layers.Conv2D(hp_units_conv3, (3, 3), activation=hp.Choice("activation6", ["relu", "tanh", "sigmoid"])))
    # model.add(tf.keras.layers.MaxPooling2D(2,2))
    model.add(tf.keras.layers.Flatten())

    hp_units_dense1 = hp.Int('denseunits1', min_value=16, max_value=512, step=16)
    model.add(tf.keras.layers.Dense(hp_units_dense1, activation=act))
    model.add(tf.keras.layers.Dense(7, activation='relu'))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(hp_learning_rate), metrics=[tf.keras.metrics.MeanAbsoluteError()])
    model.summary()
    return model

In [ ]:
tuner = None
tuner = kt.Hyperband(model_builder,
                     objective='mean_absolute_error',
                     max_epochs=10,
                     factor=3,
                     directory='/',
                     project_name='robotv0.1')

tuner.search(np.array(X_train), y_train, epochs=10, validation_split=0.2)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)

Trial 21 Complete [00h 00m 13s]
mean_absolute_error: 140.14736938476562

Best mean_absolute_error So Far: 30.53862762451172
Total elapsed time: 00h 04m 47s

Search: Running Trial #22

Value             |Best Value So Far |Hyperparameter
relu              |relu              |activation7
64                |48                |convunits1
48                |96                |convunits2
192               |48                |convunits3
384               |496               |denseunits1
0.01              |0.001             |learning_rate
4                 |10                |tuner/epochs
0                 |4                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
0                 |2                 |tuner/round

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 64)      1792      
                                  

50/50 [==============================] - 4s 49ms/step - loss: 23878.7168 - mean_absolute_error: 71.3914 - val_loss: 5952.4194 - val_mean_absolute_error: 38.7263
Epoch 2/4
49/50 [============================>.] - ETA: 0s - loss: 5740.4790 - mean_absolute_error: 37.7359

In [49]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 126, 126, 112)     3136      
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 63, 63, 112)       0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 61, 61, 80)        80720     
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 30, 30, 80)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 28, 28, 256)       184576    
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 14, 14, 256)      

In [19]:
history = model.fit(np.array(X_train), y_train, epochs=10, validation_data=(np.array(X_test), y_test))

Epoch 1/10
63/63 [==============================] - 6s 57ms/step - loss: 60167.4609 - mean_absolute_error: 133.5795 - val_loss: 14144.9463 - val_mean_absolute_error: 70.9477
Epoch 2/10
63/63 [==============================] - 3s 49ms/step - loss: 6154.3232 - mean_absolute_error: 42.0733 - val_loss: 5783.6968 - val_mean_absolute_error: 35.9853
Epoch 3/10
63/63 [==============================] - 3s 48ms/step - loss: 5323.8618 - mean_absolute_error: 34.9357 - val_loss: 5731.6182 - val_mean_absolute_error: 35.9084
Epoch 4/10
63/63 [==============================] - 3s 47ms/step - loss: 5302.9795 - mean_absolute_error: 34.7406 - val_loss: 5788.8701 - val_mean_absolute_error: 35.2504
Epoch 5/10
63/63 [==============================] - 3s 46ms/step - loss: 5314.8330 - mean_absolute_error: 35.1829 - val_loss: 5702.7656 - val_mean_absolute_error: 36.0175
Epoch 6/10
63/63 [==============================] - 3s 45ms/step - loss: 5296.9126 - mean_absolute_error: 35.1078 - val_loss: 5848.1074 - val_